# Ottimizzazione Data Analytics

Una panoramica delle librerie per la manipolazione dati in Python

In [ ]:
# Setup delle dipendenze (eseguire solo se necessario)
# !pip install pyarrow pandas polars

## Pandas: Il Data Frame Standard

**Pandas** è diventato lo standard de facto per la manipolazione dei dati in Python, offrendo:

- Una sintassi intuitiva per la manipolazione di tabelle
- Funzioni per caricamento e salvataggio dati da vari formati
- Strumenti per pulizia, trasformazione e analisi dei dati
- Integrazione profonda con l'ecosistema scientifico Python

## I Limiti di Pandas

Con l'aumento delle dimensioni dei dati, Pandas presenta diverse limitazioni:

1. **Prestazioni**: implementato in Python/C, con overhead legato al GIL
2. **Memoria**: carica l'intero dataset in RAM
3. **Parallelismo**: limitato supporto per elaborazione parallela
4. **Tipo Dati**: gestione inefficiente di alcuni tipi (es. stringhe, liste)
5. **Scalabilità**: difficoltà con dataset che superano la RAM disponibile

## Alternative a Pandas

Diverse librerie sono state sviluppate per superare i limiti di Pandas:

**Polars**: libreria per DataFrame scritta in Rust, basata su Apache Arrow
- Performance nettamente superiori
- Elaborazione in parallelo di default
- Valutazione pigra (lazy evaluation) per ottimizzazione automatica
- API simile a Pandas ma più moderna

**Dask**: estende Pandas e NumPy parallelizzando operazioni su cluster
- API quasi identica a Pandas (facile transizione)
- Elaborazione distribuita
- Gestione di dataset più grandi della memoria disponibile
- Integrazione con l'ecosistema scientifico

**cuDF (RAPIDS)**: implementazione GPU-accelerated di Pandas
- Accelerazione tramite GPU NVIDIA
- API simile a Pandas
- Integrabile con ecosistema RAPIDS per machine learning su GPU

**DuckDB**: motore di database analitico per DataFrame in-process
- Esecuzione di query SQL su DataFrame
- Performance fino a 10-100x più veloci di SQLite
- Ottimizzato per analisi OLAP
- Perfetta integrazione con Pandas e Apache Arrow

## Ottimizzare Pandas

Prima di passare ad alternative, si può migliorare Pandas con alcune tecniche:

1. **Caricamento selettivo**: usare `usecols` per leggere solo le colonne necessarie
2. **Tipi di dato ottimali**: specificare `dtype` o usare `pd.Categorical` per dati categorici
3. **Chunking**: processare dati in blocchi con `chunksize`
4. **PyArrow**: utilizzare `engine='pyarrow'` per lettura/scrittura più veloce
5. **Vettorializzare**: preferire operazioni vettoriali invece di loop Python
6. **Query**: usare `df.query()` per espressioni complesse

In [ ]:
import pandas as pd

# Configurazione per la visualizzazione
pd.set_option('display.max_rows', 6)
pd.set_option("max_colwidth", 15)

In [ ]:
# Esempio di caricamento ottimizzato
# 1. Uso di usecols per selezionare solo colonne specifiche
# 2. Uso di dtype per specificare tipi esatti
# 3. Uso di engine='pyarrow' per prestazioni migliori

colonne = ['B', 'C']
tipi = {'B': 'int32', 'C': 'string'}

# Commentato per evitare l'esecuzione che consuma tempo
pd.read_csv('data.csv', 
            usecols=colonne, 
            dtype=tipi, 
            engine='pyarrow')

In [ ]:
# Esempio di caricamento con chunk per gestire dataset grandi

result_df = []
for i, chunk in enumerate(pd.read_csv("data.csv", chunksize=2)):
    print(f"Lunghezza del chunk # {i}: {len(chunk)}")
    chunk['A'] = chunk['A'].mean()
    
    result_df.append(chunk)
pd.concat(result_df, ignore_index=True)  # Unione dei chunk filtrati

# Focus su Polars

Approfondiamo ora **Polars**, una delle alternative più promettenti a Pandas:

In [ ]:
import polars as pl

# Configurazione per la visualizzazione
pl.Config().set_tbl_rows(6)

## Pandas vs Polars

| **Pandas** | **Polars** |
|------------|------------|
| Indice Row-oriented | Column-oriented (Apache Arrow) |
| Operazioni in-place | Operazioni immutabili (funzionale) |
| GIL Python limita parallelismo | Computazioni parallele in Rust |
| NA, None, NaN, ecc. | Concetto unificato di null |

## Creazione e I/O in Polars vs Pandas

In [ ]:
# Creazione DataFrame in Pandas
pandas_df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': ['a', 'b', 'c', 'd', 'e']
})

pandas_df

In [ ]:
# Creazione DataFrame in Polars
polars_df = pl.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': ['a', 'b', 'c', 'd', 'e']
})

polars_df

In [ ]:
# Tipi dati in Pandas vs Polars
print("Pandas dtypes:")
print(pandas_df.dtypes)
print("\nPolars dtypes:")
print(polars_df.dtypes)

In [ ]:
# I/O confronto: sintassi equivalente ma internamente molto diversa

# Pandas
pandas_df.to_csv("data.csv")
df_pandas = pd.read_csv("data.csv")

# Polars
polars_df.write_csv("data.csv")
df_polars = pl.read_csv("data.csv")

In [ ]:
# Con Polars si può usare anche l'API Lazy
df_lazy = pl.scan_csv("data.csv")
df_lazy

## Principali Differenze: Context e Expressions

Polars introduce due concetti fondamentali per manipolare i dati:

1. **Context**: in quale contesto un'espressione viene valutata
2. **Expressions**: operazioni su colonne definite tramite API fluida

### I Tre Context di Polars:

1. **Selection**: `df.select(...)` e `df.with_columns(...)`
2. **Filtering**: `df.filter(...)`
3. **Aggregation**: `df.group_by(...).agg(...)`

Ogni contesto interpreta le expressions in modo diverso.

## Pandas vs Polars: Selezione Colonne

In [ ]:
# Pandas: selezione colonne
pandas_df[['A', 'B']]  # Sintassi classica
# Alternativa: pandas_df.loc[:, ['A', 'B']]

In [ ]:
# Polars: selezione colonne
polars_df.select(['A', 'B'])  # Sintassi esplicita
# Alternativa: polars_df.select(pl.col(['A', 'B']))

## Pandas vs Polars: Trasformazione Colonne

In [ ]:
# Pandas: aggiungere/modificare colonne
pandas_df_mod = pandas_df.copy()
pandas_df_mod['D'] = pandas_df_mod['A'] * 100  # Nuova colonna
pandas_df_mod['B'] = pandas_df_mod['B'] + 5    # Modifica colonna esistente
pandas_df_mod

In [ ]:
# Polars: aggiungere/modificare colonne
polars_df.with_columns([
    pl.col("A") * 100,         # Sovrascrive "A"
    (pl.col("B") + 5).alias("B"),  # Sovrascrive "B"
    pl.col("A").mul(100).alias("D")  # Nuova colonna
])

## Pandas vs Polars: Filtri

In [ ]:
# Pandas: filtri con operatori booleani
pandas_df[(pandas_df['A'] > 2) & (pandas_df['B'] <= 40)]
# Alternativa: pandas_df.query("A > 2 & B <= 40")

In [ ]:
# Polars: filtri con context dedicato
polars_df.filter(
    (pl.col("A") > 2) & (pl.col("B") <= 40)
)

## Pandas vs Polars: Aggregazioni

In [ ]:
# Dati più adatti per gruppi
df_agg = pd.DataFrame({
    'gruppo': ['A', 'A', 'A', 'B', 'B', 'C'],
    'valore1': [10, 20, 15, 30, 25, 40],
    'valore2': [100, 110, 120, 200, 210, 300]
})

df_agg

In [ ]:
# Pandas: aggregazione con groupby
df_agg.groupby('gruppo').agg({
    'valore1': ['sum', 'mean'],
    'valore2': ['min', 'max']
})

In [ ]:
# Polars: stesso dataset
pl_agg = pl.from_pandas(df_agg)

# Aggregazione con context dedicato
pl_agg.group_by('gruppo').agg([
    pl.col('valore1').sum().alias('valore1_sum'),
    pl.col('valore1').mean().alias('valore1_mean'),
    pl.col('valore2').min().alias('valore2_min'),
    pl.col('valore2').max().alias('valore2_max')
])

## Vantaggi Esclusivi di Polars

### 1. Gestione Nativa delle Liste (No Object dtype)

Pandas rappresenta liste come Python objects, causando inefficienze. Polars ha supporto nativo.

In [ ]:
# Liste in Polars
pl.DataFrame({
    'ID': [1, 2, 3],
    'valori': [[1,2,3], [4,5], [6,7,8,9]]
})

In [ ]:
# Operazioni su liste
df_list = pl.DataFrame({
    'ID': [1, 2, 3],
    'valori': [[1,2,3], [4,5], [6,7,8,9]]
})

df_list.with_columns([
    pl.col('valori').list.len().alias('lunghezza'),
    pl.col('valori').list.sum().alias('somma')
])

### 2. Lazy Evaluation

In [ ]:
# Polars: Lazy API con ottimizzazione automatica
lazy_query = (
    pl_agg.lazy()
    .filter(pl.col('valore1') > 15)
    .group_by('gruppo')
    .agg([
        pl.col('valore1').sum().alias('valore1_sum'),
        pl.col('valore2').mean().alias('valore2_mean')
    ])
    .sort('valore1_sum', descending=True)
)

# Visualizza il piano di query invece di eseguirlo
lazy_query

In [ ]:
# Esecuzione della query ottimizzata
lazy_query.collect()

In [ ]:
# Visualizzazione grafica del piano di esecuzione
lazy_query.show_graph()

### 3. Operazioni Orizzontali e Folds

In [ ]:
import polars.selectors as cs

# Aggregazione orizzontale di valori
pl_agg.select(
    pl.sum_horizontal(cs.numeric()).alias("somma_orizzontale")
)

In [ ]:
# Fold (riduzione personalizzata)
pl_agg.select(
    pl.fold(
        acc=pl.lit(1),
        function=lambda acc, x: acc * x,
        exprs=pl.col(['valore1', 'valore2'])
    ).alias('prodotto_valori')
)

## Pandas vs Polars: Join

In [ ]:
customers = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Carol', 'Dave', 'Eve']
})

orders = pd.DataFrame({
    'order_id': [101, 102, 103, 104, 105],
    'customer_id': [3, 1, 5, 2, 1],
    'amount': [50, 25, 75, 40, 15]
})

# Pandas join (usando merge)
pd_joined = customers.merge(
    orders, 
    on='customer_id', 
    how='inner'
)

pd_joined

In [ ]:
pl_customers = pl.from_pandas(customers)

pl_orders = pl.from_pandas(orders)

# Join in Polars (più esplicito)
pl_joined = pl_customers.join(
    pl_orders,
    on='customer_id',
    how='inner'
)

pl_joined

## Pandas vs Polars: Gestione Stringhe

In [ ]:
# DataFrame con stringhe
text_df = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'text': ['hello world', 'data science', 'pandas vs polars', 'python is great']
})

# Manipolazione stringhe in Pandas
text_df['uppercase'] = text_df['text'].str.upper()
text_df['words'] = text_df['text'].str.split()
text_df['word_count'] = text_df['text'].str.count(' ') + 1

text_df

In [ ]:
# DataFrame equivalente in Polars
pl_text = pl.DataFrame({
    'id': [1, 2, 3, 4],
    'text': ['hello world', 'data science', 'pandas vs polars', 'python is great']
})

# Manipolazione stringhe in Polars
pl_text.with_columns([
    pl.col('text').str.to_uppercase().alias('uppercase'),
    pl.col('text').str.split(' ').alias('words'),
    (pl.col('text').str.count_matches(' ') + 1).alias('word_count')
])

## Pipeline di Trasformazioni

In [ ]:
# Esempio di pipeline in Pandas
pd_result = (
    df_agg
    .query('valore1 > 15')
    .groupby('gruppo')
    .agg(
        valore1_sum=('valore1', 'sum'),
        valore2_mean=('valore2', 'mean')
    )
    .sort_values('valore1_sum', ascending=False)
    .reset_index()
)

pd_result

In [ ]:
# Esempio di pipeline in Polars (API Eager)
pl_result = (
    pl_agg
    .filter(pl.col('valore1') > 15)
    .group_by('gruppo')
    .agg([
        pl.col('valore1').sum().alias('valore1_sum'),
        pl.col('valore2').mean().alias('valore2_mean')
    ])
    .sort('valore1_sum', descending=True)
)

pl_result